<h1 align=center><font size = 5>Finding your ideal Neighborhood for a home in Toronto City</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Introduction Section</a>

2. <a href="#item2">Data Section</a>

3. <a href="#item3">Methodology Section</a>

4. <a href="#item4">Results Section</a>
    
5. <a href="#item4">Observation Section</a>
    
6. <a href="#item4">Conclusion Section</a>    

</font>
</div>

### Introduction where you discuss the business problem and who would be interested in this project.

People with kids, coming from suburbs, small towns or other countries to Toronto city looking for an accommodation have one common problem i.e. finding a good neighborhood for a home. Their first criterion is having a good school in the neighborhood. And then other amenities like Restaurants and Parks etc. 

This problem is faced by families all over the world to varying extents. Most impacted are the families with kids who have to be more aware of the neighborhood. In some areas, school is the primary driver while in others public transport may be the primary driver.
So this project is intended to guide one to solve this type of problem. Depending on the location and the influencing drivers this project can be modified.

### Data where you describe the data that will be used to solve the problem and the source of the data.

The dataset will be derived from different sources:
--> Location data for Toronto will be used from the previous assignments where we got the postal codes, latitudes and longitudes for Toronto city.

In [34]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")].sort_values('Postalcode').reset_index(drop=True)
neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


--> Venue data for Parks and Restaurants will be extracted using Foursquare API's using the location latitudes and longitudes.

In [43]:
toronto_parks_list.head()

,id,name,categories,neighborhood
0,4dbc8fe96a23e294ba3237bd,Glen Stewart Park,Park,The Beaches
1,51e19355498e102728a023e2,glen manor park,Park,The Beaches
2,4fb19f6ce4b0b9253b201145,Small Park,Park,The Beaches
2,4bf079c06f8aa59336f2c13c,Withrow Park,Park,"The Danforth West, Riverdale"
9,4bd4a31a5631c9b633cea230,Withrow Park Clubhouse,Park,"The Danforth West, Riverdale"


In [42]:
toronto_restaurants_list.head()

,id,name,categories,neighborhood
0,4dbc53541e72b351caafda72,Seaspray Restaurant,Asian Restaurant,The Beaches
0,4b6e0665f964a5204ea42ce3,Megas Restaurant,Greek Restaurant,"The Danforth West, Riverdale"
1,4ad89921f964a520b01221e3,Katsu Japanese Restaurant,Sushi Restaurant,"The Danforth West, Riverdale"
2,4de024f0b0fbe2cfa5fee3c4,Florentina's Italian Restaurant,Italian Restaurant,"The Danforth West, Riverdale"
3,5269ba3c498ea63de3eb987b,Simone's Caribbean Restaurant,Caribbean Restaurant,"The Danforth West, Riverdale"


--> School data will be extracted from a website. Initial idea was to scrape the HTML data from this website for all postal codes. But this site provides postal code only as an internal filter without any change in the URL. So finally the approach taken was to download data for each postal code one at a time and then merge all the postal codes in one file. http://ontario.compareschoolrankings.org/elementary/SchoolsByRankLocationName.aspx?schooltype=elementary

In [33]:
school_ratings = pd.read_csv("toronto_schools.csv")
school_ratings.head()

,Rank,Rating,Postalcode,School
0,165/3064,8.4,M4E,Balmy Beach
1,224/3064,8.2,M4E,St Denis
2,567/3064,7.5,M4E,St John
3,770/3064,7.2,M4E,Williamson Road
4,986/3064,6.9,M4E,Adam Beck


### Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

The aim was to find few of the important venues in a neighborhood that can impact a home buyer or renter, who in our case is a family with kids. I didn't consider population, size of neighborhood, crime rate etc. for this project.

Data analysis was done to understand:
--> What type of venues should be considered? For families with kids, school is a very important criteria. Next I checked for Hospitals and Parks using Foursquare and found that there are a few around most neighborhoods. So I narrowed down my search to Grocery Shops and Restaurants where data was more random. The url used for Foursquare APU included a search query for the type of venue (Park or Restaurant):

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            VERSION, 
            search_query, 
            radius, 
            LIMIT)

--> For schools, using foursquare I could not get the full list of elementary schools along with their ratings in Toronto. So I had to use a website to download data for schools by postal code. Since most families want schools with good ratings, i filtered the school data on rating >=8.
This data was then grouped on neighborhood. And this list of 16 neighborhoods is the driving list as School is our primary driver.

Inferential statistical testing: 
I wanted to include count as well as the mean rating for restaurants and grocery shops. Using Foursquare I checked for a random sample, and found that most of them don't have a rating on Foursquare. So I chose to only include only counts for Restaurant and Park in the dataset using Foursquare.

The Schools data was merged with Restaurants and Parks data using an outer join so as to keep all school neighborhoods. I used kmeans clustering to group neighborhoods based on these criteria. School Ratings, Park counts, Restaurant counts grouped by neighborhood were passed to the clustering algorithm to give the results in the form of 4 different clusters. This algorithm was best suited for this kind of clustering problem as per the dataset.


In [38]:
toronto_parks_count.head()

,Neighborhood,id,name,categories
0,"Adelaide, King, Richmond",4,4,4
1,Berczy Park,4,4,4
2,"Brockton, Exhibition Place, Parkdale Village",5,5,5
3,Business reply mail Processing Centre969 Eastern,1,1,1
4,"Cabbagetown, St. James Town",3,3,3


In [39]:
toronto_restaurant_count.head()

,Neighborhood,id,name,categories
0,"Adelaide, King, Richmond",30,30,30
1,Berczy Park,14,14,14
2,"Brockton, Exhibition Place, Parkdale Village",4,4,4
3,"Cabbagetown, St. James Town",9,9,8
4,Central Bay Street,30,30,30


In [19]:
# Only select schools where rating >= 8
top_schools = school_ratings.loc[school_ratings['Rating'] >= 8 ].reset_index(drop=True)
print(top_schools.shape)
top_schools.head()

(29, 4)


,Rank,Rating,Postalcode,School
0,165/3064,8.4,M4E,Balmy Beach
1,224/3064,8.2,M4E,St Denis
2,27/3064,9.7,M4K,Pape Avenue
3,65/3064,9.0,M4K,Jackman Avenue
4,101/3064,8.7,M4K,Withrow Avenue


In [45]:
print(schools_grouped.shape)
schools_grouped.head()

(16, 4)


,Neighborhood,Rating,Latitude,Longitude
0,Christie,8.2,43.669542,-79.422564
1,Davisville,8.2,43.704324,-79.388790
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",9.8,43.686412,-79.400049
3,"Dovercourt Village, Dufferin",10.0,43.669005,-79.442259
4,Lawrence Park,9.1,43.728020,-79.388790


### Results section where you discuss the results.

The idea is to group neighborhoods that have similar characteristics i.e. schools followed by grocery shops and restaurants. There are 38 neighborhoods in Toronto containing Toronto in the Borough name. On exploring venues using Foursquare, I see that there are 34 neighborhoods with Parks and 30 neighborhoods with Restaurants in a 500 meter radius around the neighborhood location. For Parks and Restaurants, I group by neighborhood to get the counts.

Since the first criteria is School, we narrow down our list to the neighborhoods having good elementary schools. For this I check elementary school in all 38 neighborhoods using the school data and find 29 elementary schools with rating >= 8. Next I group these schools by neighborhood to get the mean rating. Finally I have 16 neighborhoods with good schools to looks at. 

Now I merge these 16 neighborhoods with the Parks and Restaurants dataset to arrive at our final data for clustering. Among these 16 neighborhoods, I have few with either no parks or no restaurants.

Finally I run kmeans clustering on this final dataset with a cluster size of 4. The kmeans clustering algorithm does a good job in clustering the neighborhoods based on schools, parks and restaurants. The results show that majority of neighborhoods i.e. 8 out of 16 have the same characteristics. They have good schools, few parks and very few restaurants. The next two big clusters have 3 neighborhoods each. School ratings are almost the same whereas one has fewer parks and restaurants than the other. The smallest cluster has only one neighborhood with a very high school rating and a lot of restaurants. Based on these clusters, one can recommend a neighborhood to a family or let the family choose a neighborhood based on this result set.


### Merge the park and restaurant data with school data. School has the first priority so we only need to have the neighborhoods where schools are good

In [29]:
df = df[['Neighborhood','Latitude','Longitude','Rating','id_x','id_y']]
df.columns = ['Neighborhood','Latitude','Longitude','SchoolRating','ParksCount','RestaurantCount']
df.fillna(0, inplace = True)
df

,Neighborhood,Latitude,Longitude,SchoolRating,ParksCount,RestaurantCount
0,Christie,43.669542,-79.422564,8.200000,2.0,3.0
1,Davisville,43.704324,-79.388790,8.200000,3.0,6.0
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,9.800000,1.0,4.0
3,"Dovercourt Village, Dufferin",43.669005,-79.442259,10.000000,6.0,1.0
4,Lawrence Park,43.728020,-79.388790,9.100000,1.0,0.0
5,"Little Portugal, Trinity",43.647927,-79.419750,8.400000,4.0,8.0
6,"Moore Park, Summerhill East",43.689574,-79.383160,8.566667,2.0,0.0
7,North Toronto West,43.715383,-79.405678,9.300000,2.0,2.0
8,"Parkdale, Roncesvalles",43.648960,-79.456325,8.200000,0.0,4.0
9,Rosedale,43.679563,-79.377529,8.700000,3.0,0.0


In [31]:
# add clustering labels
df['Cluster Labels'] = kmeans.labels_

df.sort_values('Cluster Labels') # check the last columns!

,Neighborhood,Latitude,Longitude,SchoolRating,ParksCount,RestaurantCount,Cluster Labels
12,"Ryerson, Garden District",43.657162,-79.378937,9.900000,3.0,30.0,0
0,Christie,43.669542,-79.422564,8.200000,2.0,3.0,1
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,9.800000,1.0,4.0,1
8,"Parkdale, Roncesvalles",43.648960,-79.456325,8.200000,0.0,4.0,1
3,"Dovercourt Village, Dufferin",43.669005,-79.442259,10.000000,6.0,1.0,2
4,Lawrence Park,43.728020,-79.388790,9.100000,1.0,0.0,2
6,"Moore Park, Summerhill East",43.689574,-79.383160,8.566667,2.0,0.0,2
7,North Toronto West,43.715383,-79.405678,9.300000,2.0,2.0,2
9,Rosedale,43.679563,-79.377529,8.700000,3.0,0.0,2
10,Roselawn,43.711695,-79.416936,8.450000,0.0,0.0,2


### Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

Some observations:
--> Adding more attributes like restaurant rating would have been more useful for the end user to be able to select neighborhoods with good restaurants. Since Foursquare doesn't have good ratings data in this case, would have ideally liked to use Google or other API's instead.

--> Would have liked to scrape the school data from the website, but the process is very complicated.


### Conclusion section where you conclude the report.

Observing the clusters, one can choose which neighborhoods to look at for finding a home. 
You can use this process and algorithm in other use cases as well. Ex. to find a good neighborhood for a new restaurant or a new grocery shop. Or find a neighborhood with low crime rate and population.

The major takeaway from this project is that one can customize this dataset to add more venues as well as more attributes. For ex, we can chose to add Grocery shops or Hospitals to the venue list. Similarly one can choose to include the Restaurant rating along with the count for each neighborhood. So one can have the results based on their set of criteria.
